In [1]:
import json
import os
import time
import torch
import torch.utils.data
import torch.optim as optim
from torchvision import transforms
from my_dataset import MyDataSet
from utils import split_data, create_lr_scheduler, get_params_groups, train_one_epoch, evaluate
import numpy as np

/home/lorenzo/miniconda3/envs/rfdet/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = 'cuda:0'
data_path = "../2018.01/GOLD_XYZ_OSC.0001_1024.hdf5"
snr_range = [-20, 30]
snr_step = 2
batch_size = 16
nw = min([os.cpu_count(), batch_size if batch_size > 1 else 0, 8])  # number of workers
num_classes = 24
model = "convnext"
if model == 'convnext':
    from model import convnext as create_model
elif model == "convnet":
    from models.convnet import ConvNet as create_model

In [6]:
acc = np.zeros(int((snr_range[1] - snr_range[0])/snr_step))
device = torch.device(device if torch.cuda.is_available() else "cpu")
print(f"using {device} device...")
for snr in range(snr_range[0], snr_range[1], snr_step):
    
    # create model
    net = create_model(num_classes=num_classes).to(device)
    # load model weights
    model_weight_path = f"./weights/{model}/snr{snr}.pth"
    net.load_state_dict(torch.load(model_weight_path, map_location=device))

    data_dir = os.path.dirname(os.path.abspath(data_path))
    # train_indexes, train_labels, val_indexes, val_labels
    reload_data = True
    if os.path.exists(data_dir + '/train_indexes.json') and not reload_data:
        with open(data_dir + '/val_indexes.json') as f:
            val_indexes = json.load(f)
        with open(data_dir + '/val_label.json') as f:
            val_labels = json.load(f)
    else:
        train_indexes, train_labels, val_indexes, val_labels = split_data(data_path, snr,
                                                                          ratio=[0.875, 0.125, 0.],
                                                                          test=False, one_hot=False)

    print("using valid data size: {}".format(len(val_labels)))
    data_transform = {
        "val": transforms.Compose([transforms.ToTensor()])
    }

    val_dataset = MyDataSet(hdf5_path=data_path,
                            mod_class=val_labels,
                            indexes=val_indexes,
                            transform=data_transform["val"])
    
    val_loader = torch.utils.data.DataLoader(val_dataset,
                                         batch_size=batch_size,
                                         shuffle=False,
                                         pin_memory=True,
                                         num_workers=nw)
    
    # validate
    val_loss, val_acc = evaluate(model=net,
                                 data_loader=val_loader,
                                 device=device,
                                 epoch=1)
    acc[snr - snr_range[0]] = val_acc

using cuda:0 device...
using valid data size: 12288
  0%|          | 0/768 [00:00<?, ?it/s]


RuntimeError: Given groups=1, weight of size [32, 1, 1, 2], expected input[16, 2, 32, 32] to have 1 channels, but got 2 channels instead

In [ ]:
acc